# Zonal statistics

In [ ]:
# Load raster
import rioxarray
burned = rioxarray.open_rasterio('burned.tif')
burned

In [ ]:
import geopandas as gpd
assets = gpd.read_file('assets.gpkg')
assets

## Align the CRS of the two datasets

In [ ]:
burned.rio.crs

In [ ]:
assets.crs

In [ ]:
assets = assets.to_crs(burned.rio.crs)

# Rasterizing the vector data

In [ ]:
geom = assets[['geometry', 'code']].values.tolist()
geom

In [ ]:
burned.shape

In [ ]:
# A squeeze is needed to make the raster 2D
# Since rasterize function only accept 2D shape
burned.squeeze().shape

In [ ]:
burned_squeeze = burned.squeeze()

In [ ]:
from rasterio import features
assets_rasterized = features.rasterize(geom, out_shape=burned_squeeze.shape, transform=burned.rio.transform())

In [ ]:
assets_rasterized

## Create an Xarray.DataArray from the rasterized results

In [ ]:
import numpy as np
print(assets_rasterized.shape)
print(np.unique(assets_rasterized))

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(assets_rasterized)
plt.colorbar()

In [ ]:
assets_rasterized_xarr = burned_squeeze.copy()
assets_rasterized_xarr.data = assets_rasterized

# visualize zones
assets_rasterized_xarr.plot()

## Perform Zonal Statistcs

In [ ]:
from xrspatial import zonal_stats
stats = zonal_stats(assets_rasterized_xarr, burned_squeeze)

In [ ]:
stats

Exercise: Zonal stats over slope zones

In the previouse section, we calculated the slope over Rhodes Island. Can you perform zonal statistics over slope and non slope region? We define non slope area as `slope<0.3`.

Hint:
1. Load slope data from 'slope_dask.tif'.
2. Convert slope data to zones using [`xr.where`](https://docs.xarray.dev/en/stable/generated/xarray.where.html) function.
3. Note that the slope data and burn index data are in different resolution.

In [ ]:
# Load data and remove redundant dimension
slope = rioxarray.open_rasterio('slope_dask.tif').squeeze()

# Covert slope data to Zone data
import xarray as xr
slope2 = xr.where(slope.isnull(), 0, slope) # Set all NaN values to 0
slope_zones = xr.where(slope2 < 0.3, 0, 1)
slope_zones.rio.set_crs(slope.rio.crs)

# Reproject burned to slope
burned = rioxarray.open_rasterio('burned.tif').squeeze()
burned_match = burned.rio.reproject_match(slope_zones)

# Compute Zonal stats
stats = zonal_stats(slope_zones, burned_match)